In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

In [ ]:
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
%run driving_data.ipynb

In [ ]:
!ls

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

In [ ]:
%run model.ipynb

In [ ]:
from tensorflow.core.protobuf import saver_pb2

In [ ]:
LOGDIR = './save'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V2)

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

In [ ]:
epochs = 30
batch_size = 32

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(num_images/batch_size)):
    xs, ys = LoadTrainBatch(batch_size)
    train_step.run(feed_dict={x: xs, y_: ys, keep_prob: 0.8})
    if i % 10 == 0:
      xs_train, ys_train = LoadTrainBatch(batch_size)
      train_loss = loss.eval(feed_dict={x:xs_train, y_: ys_train, keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Training Loss: %g" % (epoch, epoch * batch_size + i, train_loss))
      xs_val, ys_val = LoadValBatch(batch_size)
      val_loss = loss.eval(feed_dict={x:xs_val, y_: ys_val, keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Validation Loss: %g" % (epoch, epoch * batch_size + i, val_loss))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

# calculate and print testing loss
test_loss = 0.0
num_test_batches = int(num_test_images/batch_size)

for i in range(num_test_batches):
  xs_test, ys_test = LoadTestBatch(batch_size)
  loss_value = loss.eval(feed_dict={x:xs_test, y_: ys_test, keep_prob: 1.0})
  test_loss += loss_value

test_loss /= num_test_batches

print("Testing Loss: %g" % test_loss)

In [ ]:
epochs = 50
batch_size = 32

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(num_images/batch_size)):
    xs, ys = LoadTrainBatch(batch_size)
    train_step.run(feed_dict={x: xs, y_: ys, keep_prob: 0.8})
    if i % 10 == 0:
      xs_train, ys_train = LoadTrainBatch(batch_size)
      train_loss = loss.eval(feed_dict={x:xs_train, y_: ys_train, keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Training Loss: %g" % (epoch, epoch * batch_size + i, train_loss))
      xs_val, ys_val = LoadValBatch(batch_size)
      val_loss = loss.eval(feed_dict={x:xs_val, y_: ys_val, keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Validation Loss: %g" % (epoch, epoch * batch_size + i, val_loss))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

# calculate and print testing loss
test_loss = 0.0
num_test_batches = int(num_test_images/batch_size)

for i in range(num_test_batches):
  xs_test, ys_test = LoadTestBatch(batch_size)
  loss_value = loss.eval(feed_dict={x:xs_test, y_: ys_test, keep_prob: 1.0})
  test_loss += loss_value

test_loss /= num_test_batches

print("Testing Loss: %g" % test_loss)

In [ ]:
epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(num_images/batch_size)):
    xs, ys = LoadTrainBatch(batch_size)
    train_step.run(feed_dict={x: xs, y_: ys, keep_prob: 0.8})
    if i % 10 == 0:
      xs_train, ys_train = LoadTrainBatch(batch_size)
      train_loss = loss.eval(feed_dict={x:xs_train, y_: ys_train, keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Training Loss: %g" % (epoch, epoch * batch_size + i, train_loss))
      xs_val, ys_val = LoadValBatch(batch_size)
      val_loss = loss.eval(feed_dict={x:xs_val, y_: ys_val, keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Validation Loss: %g" % (epoch, epoch * batch_size + i, val_loss))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

# calculate and print testing loss
test_loss = 0.0
num_test_batches = int(num_test_images/batch_size)

for i in range(num_test_batches):
  xs_test, ys_test = LoadTestBatch(batch_size)
  loss_value = loss.eval(feed_dict={x:xs_test, y_: ys_test, keep_prob: 1.0})
  test_loss += loss_value

test_loss /= num_test_batches

print("Testing Loss: %g" % test_loss)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs